In [1]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 1582377658696231737, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 16185556992
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 6332918656603514006
 physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"]

In [22]:
!pip install imgaug --upgrade

     |████████████████████████████████| 948 kB 10.0 MB/s 
  Attempting uninstall: imgaug
    Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.4.0 which is incompatible.


In [3]:
# GPU 환경 설정
import os
# os.environ["CUDA_VISIBLE_DEVICES"]="0"

# 파일경로 설정
import json

# 데이터 보기
import pandas as pd
import numpy as np
from glob import glob

# 이미지데이터 로딩
import cv2
from tqdm import tqdm

# Others
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter('ignore')

In [4]:
'''Generator'''
from tensorflow.keras.utils import Sequence
from google.colab.patches import cv2_imshow
from imgaug import augmenters as iaa
from imgaug.augmentables.batches import UnnormalizedBatch

class Generator(Sequence):
    def __init__(self, src_path, input_shape, batch_size, augs=None, is_train=True):
        
        new_image_directory = src_path + '/new_images'
        new_train_image_directory = new_image_directory + '/train'

        action_information = pd.read_csv(src_path + '/action_information.csv')

        classes = pd.get_dummies(action_information[['Label']], columns = ['Label']).to_numpy()
        
        new_train_image_directories = sorted(glob(new_train_image_directory + '/*'), key = lambda x : int(x.split('/')[-1].split('_')[-1]))

        train_answer = []
        train_image_directories = sorted(glob(new_train_image_directory + '/*'), key = lambda x : int(x.split('_')[-1]))
        for train_image_directory in train_image_directories : 
            json_path = glob(train_image_directory + '/*.json')[0]
            js = json.load(open(json_path))
            action = js.get('action')
            train_answer.append(action)
        
        self.class_num = len(classes)
        self.is_train = is_train
        self.x_data = []
        self.y_data = []
        self.batch_size = batch_size
        self.input_shape = input_shape
        self.augs = iaa.Sequential(augs)

        for new_train_image_directory, action in tqdm(zip(new_train_image_directories, train_answer), total = len(new_train_image_directories)) : 
            image_paths = sorted(glob(new_train_image_directory + '/*.jpg'), key = lambda x : int(x.split('/')[-1].replace('.jpg','')))
            image_len = len(image_paths)
            self.x_data +=image_paths
            
            for i in range(image_len):
                self.y_data.append(classes[action])

        self.on_epoch_end()
        print("Generator Initialized!!")
    
    def __len__(self):
        return round(len(self.x_data) / self.batch_size)
    
    def __getitem__(self, idx):
        batch_x, batch_y = [], []
        batch_index = self.index[idx * self.batch_size:(idx+1)*self.batch_size]
        
        for batch_i in batch_index:
            batch_x.append(self.x_data[batch_i])
            batch_y.append(self.y_data[batch_i])
        
        out_x, out_y = self.data_gen(batch_x, batch_y)
        return out_x, out_y

    def on_epoch_end(self):
        self.index = np.arange(len(self.x_data))
        if self.is_train:
            np.random.shuffle(self.index)
    
    def data_gen(self, x ,y):
        input_x = np.zeros((self.batch_size, self.input_shape[0], self.input_shape[1], 3), dtype=np.float)
        input_y = np.zeros((self.batch_size, self.class_num), dtype=np.float)

        imgs = []
        for i in range(len(x)):
            img = cv2.imread(x[i])
            imgs.append(cv2.resize(img, (self.input_shape[0], self.input_shape[1])))
        
        batch_imgs = UnnormalizedBatch(images=imgs, data=y)
        batch_aug_imgs = list(self.augs.augment_batches(batches=batch_imgs))

        for i in range(len(x)):
            aug_img = batch_aug_imgs[0].images_aug[i]
            input_x[i]= aug_img.astype(np.float) / 255.0
            input_y[i] = y[i]
        return input_x, input_y
        


In [5]:
from tensorflow.keras.applications import EfficientNetB2 # 모델은 가벼운 모델을 사용합니다.
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model

def buildModel():
    baseModel = EfficientNetB2(input_shape = (224,224,3), weights='imagenet', include_top=False, )
    baseModel.trainable = True
    model_in = Input(shape = (224,224,3))
    base_model = baseModel(model_in)
    head_model = MaxPooling2D(pool_size=(7, 7))(base_model)
    head_model = Flatten(name="flatten")(head_model)
    head_model = Dense(256, activation = 'relu')(head_model)
    head_model = Dropout(0.2)(head_model)
    head_model = Dense(32, activation = 'relu')(head_model)
    head_model = Dropout(0.2)(head_model)
    head_model = Dense(8, activation = 'relu')(head_model)
    head_model = Dropout(0.2)(head_model)
    model_out = Dense(6, activation="softmax")(head_model)

    model = Model(inputs=model_in, outputs=model_out)
    return model

In [6]:
import imgaug.augmenters as iaa

augs =[
       iaa.Fliplr(0.5),
       
        iaa.Sometimes(0.2,iaa.Sharpen()),
       
        iaa.SomeOf((1,2),[
            iaa.MultiplyAndAddToBrightness(),
            iaa.GammaContrast()
        ]),

        iaa.SomeOf((0,1), [
            iaa.Sometimes(0.7, iaa.AdditiveGaussianNoise()),
            iaa.Sometimes(0.7, iaa.GaussianBlur())
        ]),

        iaa.SomeOf((0,8),[
            iaa.ShearX(),
            iaa.ShearY(),
            iaa.ScaleX(),
            iaa.ScaleY(),
            iaa.TranslateX(),
            iaa.TranslateY(),
            iaa.Sometimes(0.3, iaa.Affine()),
            iaa.Sometimes(0.3, iaa.PerspectiveTransform()),
        ]),

        iaa.SomeOf((0,1),[
            iaa.Sometimes(0.6, iaa.Dropout()),
            iaa.Sometimes(0.6, iaa.CoarseDropout()),
            iaa.Sometimes(0.6, iaa.Cutout())
        ])
]

In [ ]:
# Modeling
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

'''Train options'''
src_path = '/gdrive/MyDrive/dacon/handSignal'
batch_size = 64
input_shape = (224, 224)
epochs = 2000
lr = 1e-3
model_name = "effient"
filename = src_path+"/save_weights/%s_{epoch:05d}.h5"%(model_name)

callback = [ReduceLROnPlateau(monitor='loss', factor=0.1, patience=3),
            ModelCheckpoint(filename, monitor="loss", save_best_only=True)]

'''Build Model'''
with tf.device('/device:GPU:0'):
    model = buildModel()

    model.compile(loss='categorical_crossentropy',optimizer=SGD(learning_rate=lr, momentum=0.9), metrics=['accuracy'])


    '''Train'''
    train_gen = Generator(src_path, input_shape, batch_size, augs=augs, is_train=True)
    model.fit_generator(train_gen, epochs=epochs, max_queue_size=50, workers=32, callbacks=callback)

100%|██████████| 142/142 [00:00<00:00, 494.81it/s]


Generator Initialized!!
Epoch 1/2000
347/347 [==============================] - 244s 640ms/step - loss: 1.8159 - accuracy: 0.1769
Epoch 2/2000
 49/347 [===>..........................] - ETA: 3:12 - loss: 1.7931 - accuracy: 0.1712

In [ ]:
!nvidia-smi

In [ ]:
'''for test generator'''
src_path = '/gdrive/MyDrive/dacon/handSignal'
batch_size = 1
input_shape = (224, 224)
gen = Generator(src_path, input_shape, batch_size, is_train=True)
gen.__getitem__(0)

# Prediction

In [14]:
src_path = '/gdrive/MyDrive/dacon/handSignal'

new_test_image_directory = src_path + '/new_images/test'
new_test_image_directories = sorted(glob(new_test_image_directory + '/*'), key = lambda x : int(x.split('file_')[-1]))
model = buildModel()
model.load_weights(src_path+"/save_weights/effient_00049.h5")

predictions = []
for new_test_image_directory in tqdm(new_test_image_directories, total = len(new_test_image_directories)) :
    image_paths = sorted(glob(new_test_image_directory + '/*.jpg'), key = lambda x : int(x.split('/')[-1].replace('.jpg','')))
    image_len = len(image_paths)
    test_images  = []
    for image_path in image_paths:
        img = cv2.imread(image_path)
        img = cv2.resize(img, (224, 224))
        img = img/255
        test_images.append(img)
    prediction = np.mean(model.predict(np.array(test_images)), axis = 0)
    predictions.append(prediction)

100%|██████████| 45/45 [33:14<00:00, 44.31s/it]


In [16]:
sample_submission = pd.read_csv(src_path + '/sample_submission.csv')
sample_submission.iloc[:,1:] = predictions
display(sample_submission.head())
sample_submission.to_csv(src_path+'/BASELINE.csv', index=False)

,file_path,Label_0,Label_1,Label_2,Label_3,Label_4,Label_5
0,./test\file_142,1.870417e-06,0.000014,0.681942,0.294948,0.022918,0.000176
1,./test\file_143,1.283897e-04,0.004398,0.080832,0.007422,0.009238,0.897982
2,./test\file_144,2.673115e-07,0.000094,0.074797,0.916459,0.008077,0.000572
3,./test\file_145,5.199605e-09,0.002207,0.884185,0.000071,0.000390,0.113147
4,./test\file_146,8.309455e-01,0.154915,0.000901,0.000104,0.011386,0.001749
